In [ ]:
# --- Colab: one-time installs ---
!pip -q install wordcloud==1.9.3

In [ ]:
# --- Imports ---
import os
import re
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from wordcloud import WordCloud
from collections import Counter

# Make outputs folder
OUT_DIR = "/content/outputs"
os.makedirs(OUT_DIR, exist_ok=True)

# Matplotlib defaults
plt.rcParams["figure.figsize"] = (10, 6)
plt.rcParams["axes.grid"] = True
plt.rcParams["figure.dpi"] = 110

print("Setup complete. Output folder:", OUT_DIR)


In [ ]:
# Load the uploaded dataset
file_path = "/content/reviews.csv"
df = pd.read_csv(file_path)

# Basic info about the dataset
info = df.info()
head = df.head()
shape = df.shape



In [ ]:
# Ensure required columns exist
required_cols = {"rating", "date", "review", "platform"}
missing = required_cols - set(df.columns.str.lower())
if missing:
    raise ValueError(f"Missing columns: {missing}. Expecting {required_cols}")

# Normalize column names
df.columns = df.columns.str.lower()

# Coerce types
df["rating"] = pd.to_numeric(df["rating"], errors="coerce")
df["review"] = df["review"].astype(str)
df["platform"] = df["platform"].str.lower().str.strip()

# Dates
df["date"] = pd.to_datetime(df["date"], errors="coerce", dayfirst=False, infer_datetime_format=True)
if df["date"].isna().all():
    # Fallback: try dayfirst True
    df["date"] = pd.to_datetime(df["date"], errors="coerce", dayfirst=True)
df["month"] = df["date"].dt.to_period("M")

print(df.dtypes)
df.sample(5)


In [ ]:
# Overall rating distribution
rating_dist = (df["rating"].value_counts(normalize=True).sort_index() * 100).round(2)
rating_dist

# Plot: Rating distribution
ax = rating_dist.plot(kind="bar")
ax.set_title("Rating Distribution (%)")
ax.set_xlabel("Rating")
ax.set_ylabel("Percentage")
for i, v in enumerate(rating_dist.values):
    ax.text(i, v + 0.5, f"{v:.1f}%", ha="center")
plt.tight_layout()
plt.savefig(f"{OUT_DIR}/rating_distribution.png", bbox_inches="tight")
plt.show()



In [ ]:
# Average rating by platform
avg_by_platform = df.groupby("platform")["rating"].mean().sort_values(ascending=False).round(3)
display(avg_by_platform)

ax = avg_by_platform.plot(kind="bar")
ax.set_title("Average Rating by Platform")
ax.set_xlabel("Platform")
ax.set_ylabel("Avg Rating")
for i, v in enumerate(avg_by_platform.values):
    ax.text(i, v + 0.03, f"{v:.2f}", ha="center")
plt.tight_layout()
plt.savefig(f"{OUT_DIR}/avg_rating_by_platform.png", bbox_inches="tight")
plt.show()

# Monthly avg rating trend (per platform)
monthly_trend = df.groupby(["month","platform"])["rating"].mean().unstack()
monthly_trend.index = monthly_trend.index.astype(str)  # for plotting
display(monthly_trend.tail())

ax = monthly_trend.plot(marker="o")
ax.set_title("Monthly Average Rating Trend by Platform")
ax.set_xlabel("Month")
ax.set_ylabel("Avg Rating")
plt.xticks(rotation=45, ha="right")
plt.tight_layout()
plt.savefig(f"{OUT_DIR}/monthly_avg_rating_trend.png", bbox_inches="tight")
plt.show()


In [ ]:
# Split by rating
pos_text = " ".join(df.loc[df["rating"] >= 4, "review"])
neg_text = " ".join(df.loc[df["rating"] <= 2, "review"])

# Generate
wc_pos = WordCloud(width=1200, height=600, background_color="white").generate(pos_text)
wc_neg = WordCloud(width=1200, height=600, background_color="black", colormap="Reds").generate(neg_text)

# Plot & save
fig = plt.figure(figsize=(16,6))
plt.imshow(wc_pos, interpolation="bilinear")
plt.axis("off")
plt.title("Positive Reviews – WordCloud")
plt.tight_layout()
plt.savefig(f"{OUT_DIR}/wordcloud_positive.png", bbox_inches="tight")
plt.show()

fig = plt.figure(figsize=(16,6))
plt.imshow(wc_neg, interpolation="bilinear")
plt.axis("off")
plt.title("Negative Reviews – WordCloud")
plt.tight_layout()
plt.savefig(f"{OUT_DIR}/wordcloud_negative.png", bbox_inches="tight")
plt.show()


In [ ]:
# Define complaint keywords (edit as needed)
keywords = ["refund", "support", "late", "delay", "cancel", "money", "issue", "delivery", "wrong", "payment", "quality"]

# Overall counts
overall_counts = {kw: df["review"].str.lower().str.contains(kw).sum() for kw in keywords}
overall_df = pd.DataFrame({"Keyword": keywords,
                           "Count": [overall_counts[k] for k in keywords]})
overall_df["Percentage"] = (overall_df["Count"] / len(df) * 100).round(2)
overall_df = overall_df.sort_values("Count", ascending=False).reset_index(drop=True)
display(overall_df)

overall_df.to_csv(f"{OUT_DIR}/complaints_overall.csv", index=False)

# Platform-wise counts
plat_stats = {}
for plat, sub in df.groupby("platform"):
    sub_text = sub["review"].str.lower()
    plat_stats[plat] = {kw: sub_text.str.contains(kw).sum() for kw in keywords}
plat_df = pd.DataFrame(plat_stats).T.fillna(0).astype(int)
plat_df["Total Reviews"] = df.groupby("platform").size()
for kw in keywords:
    plat_df[f"{kw}_pct"] = (plat_df[kw] / plat_df["Total Reviews"] * 100).round(2)

display(plat_df)
plat_df.to_csv(f"{OUT_DIR}/complaints_by_platform.csv", index=True)


In [ ]:
# Plot a focused comparison for the top 5 complaint categories
key_complaints = ["refund", "support", "cancel", "delivery", "money"]
plot_df = plat_df[key_complaints].div(plat_df["Total Reviews"], axis=0) * 100

ax = plot_df.plot(kind="bar")
ax.set_title("Top Complaint Categories (% of Reviews) by Platform")
ax.set_ylabel("Percentage of Reviews (%)")
ax.set_xlabel("Platform")
plt.xticks(rotation=0)
plt.tight_layout()
plt.savefig(f"{OUT_DIR}/complaints_by_platform.png", bbox_inches="tight")
plt.show()


In [ ]:
# Simple rule-based sentiment from rating
def label_sentiment(r):
    if r >= 4: return "Positive"
    if r == 3: return "Neutral"
    return "Negative"

df["sentiment"] = df["rating"].apply(label_sentiment)

sent_counts = df.groupby(["month","platform","sentiment"]).size().reset_index(name="count")
sent_counts["total"] = sent_counts.groupby(["month","platform"])["count"].transform("sum")
sent_counts["pct"] = (sent_counts["count"] / sent_counts["total"] * 100).round(2)
sent_counts["month"] = sent_counts["month"].astype(str)

# Save table
sent_counts.sort_values(["platform","month","sentiment"]).to_csv(f"{OUT_DIR}/sentiment_trend_table.csv", index=False)
display(sent_counts.head())

# Plot per platform
for plat in df["platform"].unique():
    temp = sent_counts[sent_counts["platform"] == plat]
    pivot = temp.pivot(index="month", columns="sentiment", values="pct").fillna(0)
    ax = pivot.plot(marker="o")
    ax.set_title(f"Sentiment Trend Over Time – {plat.capitalize()}")
    ax.set_xlabel("Month")
    ax.set_ylabel("% of Reviews")
    plt.xticks(rotation=45, ha="right")
    plt.tight_layout()
    plt.savefig(f"{OUT_DIR}/sentiment_trend_{plat}.png", bbox_inches="tight")
    plt.show()


In [ ]:
# Pull highlight metrics
total_reviews = len(df)
one_star_pct = ( (df["rating"]==1).mean() * 100 )
five_star_pct = ( (df["rating"]==5).mean() * 100 )
avg_by_plat_text = "\n".join([f"- {p}: {v:.2f} ⭐" for p, v in avg_by_platform.items()])

# Identify top 3 complaints overall
top3 = overall_df.head(3)[["Keyword","Percentage"]].values.tolist()
top3_text = "\n".join([f"- {k}: {pct:.2f}%" for k, pct in top3])

report = f"""# Quick Commerce Review Insights – Executive Summary

**Dataset:** {total_reviews:,} reviews across Blinkit, Zepto, JioMart
**Columns:** rating, date, review, platform

## Overall Sentiment
- 1★ share: **{one_star_pct:.1f}%**
- 5★ share: **{five_star_pct:.1f}%**
- Interpretation: Customer sentiment is **highly negative** overall.

## Average Rating by Platform
{avg_by_plat_text}

## Top Complaint Themes (Overall)
{top3_text}

## Visuals (see /outputs)
- rating_distribution.png
- avg_rating_by_platform.png
- monthly_avg_rating_trend.png
- complaints_by_platform.png
- wordcloud_positive.png
- wordcloud_negative.png
- sentiment_trend_<platform>.png

## Recommendations
- Blinkit: Improve delivery reliability and SLA adherence.
- JioMart: Reduce cancellations; streamline and automate refunds.
- Zepto: Strengthen customer support; fix refund/payment friction.
- All: Proactive comms, faster escalation paths, and transparent refund timelines.

*Generated automatically by the Colab notebook.*
"""

with open(f"{OUT_DIR}/REPORT.md", "w", encoding="utf-8") as f:
    f.write(report)

print(f"Report written to: {OUT_DIR}/REPORT.md")


In [ ]:
# Save a cleaned subset for sharing
clean_cols = ["rating","date","review","platform","month","sentiment"]
df[clean_cols].to_csv(f"{OUT_DIR}/reviews_clean.csv", index=False)
print("Saved:", f"{OUT_DIR}/reviews_clean.csv")
